---
layout: post
title: Blog Post 2
---

In this blog post, you’ll create a simple webapp using `Flask` and describe the how you did it. The skills you’ll need are:

- Flask fundamentals, including `render_template()`, the basic anatomy of templates, and user interactions.  
- Database skills, including adding items to databases and displaying them.  
- Basic CSS in order to add a bit of personal flare to your webapp.  

You will create your website by modifying Prof. Chodrow's [inclass template](https://github.com/PIC16B/flask-interactions). Fork this to your github account first, then open it with Github desktop and start modifying the file in your local computer.

# Create __init__.py(app.py)

We start by creating this file, and it is our main file for the website. To run this website and watch for changes, we need to set directory to our current folder. After activate PIC16B cell, type `set FLASK_ENV=development` and then `flask run` in your command window. Copy and paste the output link to your browser, you should be able to see your website.

First of all, we need some standard imports.

In [ ]:
from flask import Flask, g, render_template, request

import numpy as np
import pandas as pd
import pickle
import sqlite3

import io
import base64

app = Flask(__name__)

## Modify base.html file

We want a title and navigation links to submit and view page at the top of every webpage in our app. It is helpful to put them inside a template called `base.html`. We will extend `base.html` in `main.html`, `submit.html` and `view.html`.

```
<!doctype html>
<link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
<title>{% block title %}{% endblock %} - PIC16B Website</title>

<nav>
  <h1>PIC16B: A Simple Message Bank</h1>
  <!-- <b>Navigation:</b> -->
</nav>
<section class="content">
    <ul>
      <h1 style="font-size:1.875em; font-family: cursive;" ><li><a href="{{ url_for('submit') }}">Submit a message</a></li>
      <li><a href="{{ url_for('view') }}">View messages</a></li></h1>
    </ul>
  {% block content %}{% endblock %}
</section>
```

We have changed the original title, and added the navigation links to the content part.

## Welcome Page

We could easily create a main page by typing in this to our `.py` file. 

In [ ]:
# Create main page
@app.route('/', methods=['POST', 'GET'])
def main():
    return render_template("main.html")

This is basically calling `main.html`. In `main.html`, we will extend `base.html` to keep the title, then simply add a welcome message. 

```
{% extends 'base.html' %}
{% block content %}
<p>Welcome to this simple message bank webapp!</p>
{% endblock %}
```

Great! Now we have successfully constructed our welcome page. It should look like this(We will talk about how to change the font and color in later sections. Don't worry about it for now if your background looks different).

## Submit Page

First, create a submit template with three user interface elements:  

- A text box for submitting a message.  
- A text box for submitting the name of the user.  
- A “submit” button.  

```
{% extends 'base.html' %}


{% block content %}
<center>
<p><h2><strong>Please Submit Your Message Here</strong></h2></p>
  <form method="post">
      <label for="message">Your message: </label><br>
      <input name="message">
      <br>
      <label for="message">Your name or handle: </label><br>
      <input name="handle">
      <br>
      <input type="submit" value="Submit form">
  </form>
</center>
  {% if thanks %}
    Thank you for submitting a message!
  {% endif %}


{% endblock %}
```

We extend `base.html` and add `<center>` and `</center>` to center all messages. Also, don't forget to include a thank you message.

Now, write two Python functions for database management in a new file `app.py` (you can also write them in a separate `.py` file and import them into `app.py`).  

`get_message_db()` should handle creating the database of messages.
- Check whether there is a database called `message_db` in the `g` attribute of the app. If not, then connect to that database, ensuring that the connection is an attribute of `g`. To do this last step, write a line like do `g.message_db = sqlite3.connect("messages_db.sqlite")`.
- Check whether a table called `messages` exists in `message_db`, and create it if not. For this purpose, the SQL command `CREATE TABLE IF NOT EXISTS` is helpful. Give the table an id column (integer), a handle column (text), and a message column (text).
- Return the connection `g.message_db`.

In [ ]:
def get_message_db():
    # Check whether there is a database called message_db in the g attribute of the app
    if 'message_db' not in g:
        #  If not, then connect to that database, ensuring that the connection is an attribute of g
        g.message_db = sqlite3.connect("messages_db.sqlite")

    if g.message_db is not None:
        cursor = g.message_db.cursor()
        # Check whether a table called messages exists in message_db, and create it if not
        sql_create_messages_table = """ CREATE TABLE IF NOT EXISTS messages (
                                    id integer,
                                    handle text,
                                    messages text
                                ); """
        cursor.execute(sql_create_messages_table)
    # Return the connection
    return g.message_db

`insert_message(request)` should handle inserting a user message into the database of messages.  
- Extract the message and the handle from request. You’ll need to ensure that your `submit.html` template creates these fields from user input by appropriately specifying the name of the input elements. For example: `<input type="text" name="message" id="message">` is what I used in my template to ensure that `request.form["message"]` contained the message input by the user. You should then return the message and the handle.
- Using a cursor, insert the message into the message database. Remember that you’ll need to provide an ID number, the handle, and the message itself. You’ll need to write a SQL command to perform the insertion.
   - Note: when working directly with SQL commands, it is necessary to run `db.commit()` after inserting a row into db in order to ensure that your row insertion has been saved.
- You should ensure that the ID number of each message is unique. One way to do this is by setting the ID number of a message equal to one plus the current number of rows in message_db.
- Don’t forget to close the database connection within the function!

In [ ]:
def insert_message(request):
    # open the connection
    g.message_db = get_message_db()
    cursor = g.message_db.cursor()
    # Extract message and handle
    message = request.form["message"]
    handle = request.form["handle"]
    
    # get nrow and assign unique id
    n_row = cursor.execute('select * from messages;')
    nrow = len(n_row.fetchall()) + 1
    
    # add a new row to messages database
    cursor.execute("INSERT INTO messages (id, handle, messages) VALUES ({nrow}, '{handle}', '{message}')".format(
         nrow = nrow, handle = handle, message = message))
    # Save the change
    g.message_db.commit()
    # close the connection
    g.message_db.close()

Finally, write a function to `render_template()` the submit.html template. Since this page will both transmit and receive data, we should ensure that it supports both POST and GET methods, and give it appropriate behavior in each one. In the GET case, we can just render the `submit.html` template with no other parameters. In the POST case, we should call `insert_message()` (and then render the `submit.html` template). We will also add a small note thanking the user for their submission.

In [ ]:
@app.route('/submit/', methods=['POST', 'GET'])
def submit():
    if request.method == 'GET':
        return render_template('submit.html')
    else: # if request.method == 'POST'
        try:
            insert_message(request)
            return render_template('submit.html', thanks = True)
        except:
            return render_template('submit.html')

Now you have created your Submit Page. It should look like this in your webapp:

## View Page

To view all the submitted messages, we will need a helper function to extract them. We will write a function called random_messages(n) which will return a collection of n random messages from the message_db, or fewer if necessary. This [StackOverflow](https://stackoverflow.com/questions/2279706/select-random-row-from-a-sqlite-table) post might help. Don’t forget to close the database connection within the function!

In [ ]:
def random_messages(n):
    # open the connection
    g.message_db = get_message_db()
    # Get a collection of n random messages from the message_db, or fewer if necessary
    messages = pd.read_sql_query("SELECT * FROM messages WHERE id IN (SELECT id FROM messages ORDER BY RANDOM() LIMIT {n})".format(n = n), g.message_db)
    # close the connection
    g.message_db.close()
    return messages

Next, write a new template called `view.html` to display the messages extracted from `random_messages()`.

```
{% extends 'base.html' %}

{% block content %}
<center>
<p><h2><strong>Some Cool Messages</strong></h2></p>

{% for index, row in messages.iterrows() %}
<strong>{{ row[2]}}</strong><br>
--<em>{{row[1]}}</em><br>
<br>
{% endfor %}
</center>

{% endblock %}
```

Again, we extend `base.html`. I have added `<center>` and `</center>` to center all messages.

I took advantage of the fact that Jinja tags support looping (so I looped over the messages), and I also used the fact that Jinja tags support indexing of objects (so if m is a tuple of user handle and message m[0] contains the handle and m[1] contains the message). In this case, the second column of `messages` table is message, and the third column is handle/name.

Finally, write a function to render your `view.html` template.

In [ ]:
@app.route('/view/')
def view():
    try:
        messages = random_messages(5)
        return render_template('view.html', messages = messages)
    except:
        return render_template('view.html', error = True)

This function should first call `random_messages()` to grab some random messages (I chose a cap of 5), and then pass these messages as an argument to `render_template()`.

Now you have created your View Page. It should look like this in your webapp:

## Modify Style.css to change Webapp Appearance

You should be able to change the Webapp Appearance by modifying `style.css`. For example, to change the font family and background color, here is what I did:

```
html {
    font-family: "New Century Schoolbook";
    background: #add8e6;
    padding: 1rem;
}
```

Similarly, you can also change other settings in this file. Just change the corresponding section accordingly and you should be able to see all the changes by refreshing your Webapp. Feel free to add additional CSS (and modify your templates if necessary) in order to give your app a personal feel.

Congratulations! Now you have successfully built a simple message bank webapp!

Lastly, here is a link to my [GitHub repository](https://github.com/JadenWSR/flask-interactions) containing the code for the app.